In [1]:
import gym
import roboticsPlayroomPybullet


# Open the env

In [2]:
# Create an 
env = gym.make('UR5PlayAbsRPY1Obj-v0')
# Launch a GUI - don't call this if you want it headless
env.render('human')
# Reset the env - must be called before using the env as this initialises everything
_ = env.reset()
# Activate img observations as part of the 'o' return dict
env.render('playback')

current_dir=/home/sholto/Desktop/AI/RoboticsPlayroomPybullet/roboticsPlayroomPybullet/envs
current_dir=/home/sholto/Desktop/AI/RoboticsPlayroomPybullet/roboticsPlayroomPybullet/envs
current_dir=/home/sholto/Desktop/AI/RoboticsPlayroomPybullet/roboticsPlayroomPybullet/envs


/home/sholto/anaconda3/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


/home/sholto/Desktop/AI/RoboticsPlayroomPybullet/roboticsPlayroomPybullet/envs


# Set the path of the data dirs

In [3]:

import os
from pathlib import Path
import numpy as np


TEST_DATASET = "UR5_slow_gripper_test" 
print('Using local setup')
WORKING_PATH = Path().absolute().parent
print(f'Working path: {WORKING_PATH}')
os.chdir(WORKING_PATH)
STORAGE_PATH = WORKING_PATH
print(f'Storage path: {STORAGE_PATH}')
TRAIN_DATA_PATHS = [STORAGE_PATH/'data'/x for x in ["UR5", "UR5_high_transition", "UR5_slow_gripper"]]
TEST_DATA_PATH = STORAGE_PATH/'data'/TEST_DATASET

# TRAIN_DATA_PATHS are the big folders which represent a few hours of teleop on different days
# Within each of these folders are individual trajectories, which are occasionally reset for breaks 
# or the block goes out of reach.
# Structure
# DIR
 # DIR/obs_act_etc/  - this contains the npz files with obs/acts/ags at 25Hz, all info necessary to train models
                # /1
                # /2 etc
 # DIR/states_and_ims/ - this contains the states we can reset to (which contain info like precise contact points\
                        # which is better for deterministic reset and rollout, but large so we only saved these at > 1Hz

Using local setup
Working path: /home/sholto/Desktop/AI/learning_from_play
Storage path: /home/sholto/Desktop/AI/learning_from_play


# Iterate through the teleop data, showcasing RL support

In [5]:
import scipy.misc
from IPython.display import display, clear_output
from natsort import natsorted
import glob

# The relevant keys from the teleop data  - other keys include ['obs', 'acts', 'achieved_goals', 'joint_poses', 'target_poses', 'acts_quat', 'acts_rpy_rel',  'velocities', 'obs_quat']
# Which are useful if you want to act with joint poses, in quaternion space for obs, with velocity etc

keys = ['obs', 'acts', 'achieved_goals']
full_state_save_interval = 30 # How often we saved the determinstically resettable state (full info e.g. contact points )

for DIR in TRAIN_DATA_PATHS:
    DIR = str(DIR) # glob/natsorted uses strings
    
    obs_act_path = DIR+'/obs_act_etc/'

    # Each demo is one continuous play trajectory
    for demo in natsorted(os.listdir(obs_act_path)):
        # Gets the saved states (which we can reset to) 
        start_points = natsorted(glob.glob(DIR+'/states_and_ims/'+str(demo)+'/env_states/*.bullet'))
        # The actual data
        traj = np.load(obs_act_path+demo+'/data.npz')
        # The actions taken and the states of the non-arm elements of the environment
        acts,achieved_goals = traj['acts'], traj['achieved_goals']
        # The total length of this recorded trajectory
        set_len = len(acts)
    
        start = 0 
        # End is the steps till the next reset state, or the end of the demo
        end= min(start+full_state_save_interval, set_len)
        print(DIR+'/states_and_ims/'+str(demo)+'/ims')

        # Between each state we reset to - roll out the actions. 
        # A tiny amount of divergence is to be expected, which is why we reset whenever we have the information to do so
        for start_point in start_points:

            # As the states were saved without subgoals - delete these or we will not be able to reset state 
            env.delete_sub_goal()
            env.p.restoreState(fileName=start_point)
            env.instance.updateToggles() # need to do it when when you restore from a state as colors are not saved
            
            goal = achieved_goals[end] # gets the goal 30 timesteps away
            env.reset_goal_pos(goal) # sets the env's goal to this
            env.instance.visualise_sub_goal(goal) # visualises it using transparent objects
            for i in range(start, end):
                # Play out the teleoperated actions
                o,r,_,_ = env.step(acts[i])
                # r is sparse -1, 0 and defined in 'playRewardFunc.py' in the env repo

            start += full_state_save_interval
            end = min(start+full_state_save_interval, set_len)

/home/sholto/Desktop/AI/learning_from_play/data/UR5/states_and_ims/0/ims


KeyboardInterrupt: 

In [9]:
o

{'obs_quat': array([-1.05122246e-01, -1.58003315e-01, -3.55988974e-03, -9.47304070e-03,
        -9.51900929e-02,  1.15734883e-01,  9.88663018e-01,  5.44099584e-02,
        -9.76497903e-02,  1.10748991e-01, -9.99999975e-06, -2.50560520e-15,
         8.16640219e-15,  9.60237384e-01, -2.79184759e-01, -4.55279015e-02,
         4.82954718e-02,  2.99999993e-02,  0.00000000e+00], dtype=float32),
 'achieved_goal': array([-9.7649790e-02,  1.1074899e-01, -9.9999997e-06, -2.5056052e-15,
         8.1664022e-15,  9.6023738e-01, -2.7918476e-01, -4.5527902e-02,
         4.8295472e-02,  2.9999999e-02,  0.0000000e+00], dtype=float32),
 'desired_goal': array([-0.12080648,  0.24572381,  0.09474243], dtype=float32),
 'controllable_achieved_goal': array([-0.10512225, -0.15800332, -0.00355989,  0.05440996], dtype=float32),
 'full_positional_state': array([-1.05122246e-01, -1.58003315e-01, -3.55988974e-03, -9.47304070e-03,
        -9.51900929e-02,  1.15734883e-01,  9.88663018e-01,  5.44099584e-02,
        -9